# Proof of concept of PERM-Based Regular ISD

This code applies PERM-Based Regular ISD on random RSD instances, measures the success probability and compares it with theoretical estimates.

In particular, we expemimentally estimate the following quantities:
- probability that $H'$ (obtained from $H$ by adding the parity-checks encoding regularity) has full rank
- probability that the sampled set of $k' = k-w$ positions is not an information set
- success probability

In [9]:
reset(); 

#Generate random RSD instance (requires that n is divided by b)
def sample_rsdp_instance(n,k,b,w):

    F2 = GF(2);

    #Sample full rank parity-check matrix
    rank_H = 0;
    while rank_H < (n-k):
        H = random_matrix(F2,n-k,n);
        rank_H = rank(H);

    #Sample regular error vector
    e = matrix(F2,1,n);
    for i in range(w):
        pos = randrange(b);
        e[0,i*b+pos] = F2(1);

    #Compute syndrome
    s = H*e.transpose();

    return H, e, s;

##############################################

#Include the additional parity-check equations
def add_parity_checks(H, s, n, k, b, w):

    #Compute adapted parameters
    r = n - k;
    rp = n - k + w;
    kp = n - rp;

    ##Modify H: add partiy-check equations, remove last columns
    new_H = matrix(GF(2),rp,n);
    new_H[0:r, :] = H;
    for i in range(w):
        for j in range(b):
            new_H[r+i,i*b+j] = 1;

    new_H = new_H[:,0:n];

    ##Modify s: add new bits, corresponding to new parity-check equations
    new_s = matrix(GF(2), rp, 1);
    new_s[0:r, 0] = s;
    for i in range(w):
        new_s[r+i,0] = 1;

    return new_H, new_s;

# Implementation of PERM-Based regular ISD
The implementation is only meant to be a proof of concept; it is only meant to verify the success probability and the probabilities that the employed have the desired rank 

In [10]:
#Sample regular permutation
#Sample v_f coordinates from w_f blocks, v_c coordinates from w_c blocks and move them in first positions;
#These coordinates will constitute the information set
#The remaining coodinates are moved in the last positions

def sample_regular_permutation(n, b, v_f, v_c, w_f, w_c):
    
    P_list = []; #regular permutation
    
    #We first sample coordinates for the w_f blocks (v_f from each block)
    for i in range(w_f):
        
        perm_of_b = Permutations(range(b)).random_element();
        for j in range(v_f):
            P_list.append(i*b+perm_of_b[j]);
    
    #Now, sample coordinates for the w_c blocks (v_c from each block)
    for i in range(w_c):
        
        perm_of_b = Permutations(range(b)).random_element();
        for j in range(v_c):
            P_list.append((w_f+i)*b+perm_of_b[j]);

    #Now, place the remaining coordinates (i.e., those that will be move to the last n-k' positions)
    for i in range(n):
        if i not in P_list:
            P_list.append(i);
    
    #Create permutation matrix out of P
    P = matrix(GF(2),n,n);
    for i in range(n):
        P[P_list[i], i] = 1;
        
    return P;

######################
#Perm-based ISD; it receives as input the H with additional equations
#It also updates the empirical estimates (last three parameters)
def PERM_based_ISD(n, k_prime, w, b, new_H, new_s, num_success, num_full_rank, num_new_H_full_rank):
    
    ok = 0; #ok becomes 1 when a solution is found

    #Sample regular permutation
    P = sample_regular_permutation(n, b, v_f, v_c, w_f, w_c);

    #Apply permutation to H
    perm_H = new_H*P;

    #Do Gaussian elimination
    perm_H_left = perm_H[:,k_prime:n];

    is_full_rank = 0; #set to 1 if left-k'matrix has full rank

    if rank(perm_H_left) == (n-k_prime): #if full rank, we found an info set

        is_full_rank = 1;

        perm_e1 = new_s.transpose()*perm_H_left.inverse().transpose();
        weight_perm_e1 = perm_e1.list().count(1);

        if weight_perm_e1 == w:

            #we only need to check if the found vector yields to a regular vector
            perm_candidate_e = matrix(GF(2),1,n);
            perm_candidate_e[0,k_prime:n] = perm_e1;

            #Apply inverse permutation
            candidate_e = perm_candidate_e*P^-1;

            #Check if the found vector is regular
            is_regular = 1;
            for i in range(w):
                num_ones_in_block = candidate_e[0,i*b:(i+1)*b].list().count(1);
                if num_ones_in_block != 1:
                    is_regular = 0;

            if is_regular:
                ok = 1;  
             #   print(candidate_e==e);
                    
        num_success += ok;
        num_full_rank += is_full_rank;
        
    return num_success, num_full_rank, num_new_H_full_rank;

# Set simulation

Select the code parameters ($n$, $k$, $b$ and $w$) and the number of RSD instances to be generated (denoted by $\mathtt{num}\_\mathtt{instances}$)

In [11]:
#Parameters
n = 90; #code length
k = 45; #code dimension
w = 10; #number of blocks with weight 1

num_instances = 100000; #number of RSD instances

b = n/w; #size of blocks (length of unit vectors forming the solution)

S = max(1,b**w/(2**(n-k))); #number of solutions

k_prime = k-w; #new code dimension
p_iter = (1-k_prime/n)**w; #success probability without considering rounding issues, single solution

#We now consider rounding issues and the corresponding success probability
v_f = floor(k_prime/w);
v_c = ceil(k_prime/w);

w_f = w - (k_prime-w*v_f);  #number of blocks from which we select v_f coordinates
w_c = w - w_f; #number of blocks from which we select v_c coordinates

p_iter_rounding = (1-v_f/b)**w_f * (1-v_c/b)**w_c; #success probability with rounding issues, single solution

print("Considering: [n, k, w, b] = "+str([n, k, w, b]));
print("Number of solutions: S = "+str(N(S)));
print("Th. success probability: p_iter(NO rounding) = "+str(N(0.2887*min(1, S*p_iter)))+", p_iter(WITH rounding) = "+str(N(0.2887*min(1, S*p_iter_rounding))));
      

Considering: [n, k, w, b] = [90, 45, 10, 9]
Number of solutions: S = 1.00000000000000
Th. success probability: p_iter(NO rounding) = 0.00209724218345199, p_iter(WITH rounding) = 0.00201199993839252


# Start simulation and wait for results!

In [12]:
num_success = 0; #number of successful iterations
num_full_rank = 0; #num of full rank matrices in ISD
num_new_H_full_rank = 0; #number of H' with full ranks

for id_instance in range(1, num_instances+1):

    #Sample RSD instance
    H, e, s = sample_rsdp_instance(n, k, b, w);

    #Add parity-chek equations
    new_H, new_s = add_parity_checks(H, s, n, k, b, w);
    
    if rank(new_H)==n-k_prime:
        num_new_H_full_rank +=1;
                
    #Launch PERM-based ISD
    num_success, num_full_rank, num_new_H_full_rank = PERM_based_ISD(n, k_prime, w, b, new_H, new_s, num_success, num_full_rank, num_new_H_full_rank);
    
     #Print comparison between theoretical and estimated
    if (id_instance%10000) == 0:
        
        print("Num instances = "+str(id_instance));
        table_rows = []
        table_rows.append(["Experimental: Pr[H' has full rank]",str(N(num_new_H_full_rank/id_instance))])
        table_rows.append(["Experimental: Pr[Gaussian El. is ok]",str(N(N(num_full_rank/id_instance)))])        
        table_rows.append(["Experimental: Pr[Success]",str(N(N(num_success/id_instance)))])
        table_rows.append(["Theoretical: Pr[Success] (NO ROUNDING)",str(N(N(0.2887*min(1, S*p_iter))))])
        table_rows.append(["Theoretical: Pr[Success] (WITH ROUNDING)",str(N(N(0.2887*min(1, S*p_iter_rounding))))])
        
        t = table(table_rows, header_row = ["Probability", "Value"])
        show(t)
        
        print("------------------------------");

Num instances = 10000


Probability,Value
Experimental: Pr[H' has full rank],1.00000000000000
Experimental: Pr[Gaussian El. is ok],0.285200000000000
Experimental: Pr[Success],0.00160000000000000
Theoretical: Pr[Success] (NO ROUNDING),0.00209724218345199
Theoretical: Pr[Success] (WITH ROUNDING),0.00201199993839252


------------------------------
Num instances = 20000


Probability,Value
Experimental: Pr[H' has full rank],1.00000000000000
Experimental: Pr[Gaussian El. is ok],0.286700000000000
Experimental: Pr[Success],0.00175000000000000
Theoretical: Pr[Success] (NO ROUNDING),0.00209724218345199
Theoretical: Pr[Success] (WITH ROUNDING),0.00201199993839252


------------------------------
Num instances = 30000


...

------------------------------
Num instances = 90000


Probability,Value
Experimental: Pr[H' has full rank],1.00000000000000
Experimental: Pr[Gaussian El. is ok],0.287722222222222
Experimental: Pr[Success],0.00213333333333333
Theoretical: Pr[Success] (NO ROUNDING),0.00209724218345199
Theoretical: Pr[Success] (WITH ROUNDING),0.00201199993839252


------------------------------
Num instances = 100000


Probability,Value
Experimental: Pr[H' has full rank],1.00000000000000
Experimental: Pr[Gaussian El. is ok],0.288360000000000
Experimental: Pr[Success],0.00213000000000000
Theoretical: Pr[Success] (NO ROUNDING),0.00209724218345199
Theoretical: Pr[Success] (WITH ROUNDING),0.00201199993839252


------------------------------
